# Train NN by Iris Dataset using 10-fold cross validation

## Imports

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold

### Reset graph

In [2]:
tf.reset_default_graph()

### Load raw data

In [3]:
raw_data = load_iris()
Features, Labels = raw_data.data, raw_data.target

### Pre-processors

In [4]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
X_scaler = MinMaxScaler(feature_range=(0, 1))
one_hot_encoder = OneHotEncoder(sparse=False, categories='auto')

## Model Definition

### Define model parameters

In [5]:
learning_rate = 0.01
training_epochs = 10000

### Define layer sizes

In [6]:
layer_1_size = 50
layer_2_size = 10

### Define number of inputs and outputs

In [7]:
num_input = Features.shape[1]
num_output = len(np.unique(Labels, axis=0))

num_input, num_output

(4, 3)

### Define layers

In [8]:
with tf.variable_scope("input"):
    X = tf.placeholder(tf.float32, shape=[None, num_input])

#### Layer 1

In [9]:
with tf.variable_scope("layer_1"):
    weights = tf.get_variable(
        "weights1", 
        shape=[num_input, layer_1_size],
        initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(
        "bias1",
        shape=[layer_1_size],
        initializer=tf.zeros_initializer()
    )
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

#### Layer 2

In [10]:
with tf.variable_scope("layer_2"):
    weights = tf.get_variable(
        "weights2", 
        shape=[layer_1_size, layer_2_size],
        initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(
        "bias2",
        shape=[layer_2_size],
        initializer=tf.zeros_initializer()
    )
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

#### Output

In [11]:
with tf.variable_scope("output"):
    weights = tf.get_variable(
        "weights3", 
        shape=[layer_2_size, num_output],
        initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(
        "bias3",
        shape=[num_output],
        initializer=tf.zeros_initializer()
    )
    prediction = tf.matmul(layer_2_output, weights) + biases

#### Cost

In [12]:
with tf.variable_scope("cost"):
    Y = tf.placeholder(tf.float32, shape=[None, num_output])
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=prediction))

#### Optimize

In [13]:
with tf.variable_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#### Accuracy

In [14]:
with tf.variable_scope("accuracy"):
    corr_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(prediction, axis=1))
    accuracy = tf.reduce_mean(tf.cast(corr_pred, tf.float32))

#### Log Result

In [15]:
with tf.variable_scope("logging"):
    tf.summary.scalar("current_cost", cost)
    tf.summary.scalar("current_accuracy", accuracy)
    summary = tf.summary.merge_all()

## Running Session

### Declare utility variables

In [16]:
train_writer = tf.summary.FileWriter('assignments/neural-network/tmp/train')
test_writer = tf.summary.FileWriter('assignments/neural-network/tmp/test')

### Logger function

In [19]:
def log_progress():
    train_cost, train_summary = session.run(
        [cost, summary], feed_dict=train_feed
    )
    test_cost, test_summary = session.run(
        [cost, summary], feed_dict=test_feed
    )
    train_acc = session.run(accuracy, feed_dict=train_feed)
    test_acc = session.run(accuracy, feed_dict=test_feed)
    print(epoch, train_cost, test_cost, train_acc, test_acc)
    train_writer.add_summary(train_summary, epoch)
    test_writer.add_summary(test_summary, epoch)


### Trainer function

In [24]:
def run_training(train_feed_dict, test_feed_dict, log=False):
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        
        for epoch in range(training_epochs):
            session.run(optimizer, feed_dict=train_feed)
    
            if epoch % 1000 == 0 and log:
                log_progress()
    
        if log:
            print("Training is completed", "\n")
        final_train_acc = session.run(accuracy, feed_dict=train_feed_dict)
        final_test_acc = session.run(accuracy, feed_dict=test_feed_dict)
        
        if log:
            print("Final Train Accuracy:", final_train_acc)
            print("Final Test Accuracy:", final_test_acc)
            print("--------------------------------------")
        
        return final_train_acc, final_test_acc

### Pre-processor function

In [22]:
def pre_process(features, labels, train_index, test_index):
    X_train, X_test = features[train_index], features[test_index]
    Y_train, Y_test = labels[train_index], labels[test_index]
    X_train_scaled = X_scaler.fit_transform(X_train)
    X_test_scaled = X_scaler.fit_transform(X_test)
    Y_train_encoded = one_hot_encoder.fit_transform(Y_train.reshape((-1, 1)))
    Y_test_encoded = one_hot_encoder.fit_transform(Y_test.reshape((-1, 1)))
    
    return X_train_scaled, X_test_scaled, Y_train_encoded, Y_test_encoded

### RUN!

In [23]:
train_accuracy, test_accuracy = 0, 0

for train_index, test_index in skf.split(Features, Labels):
    X_train_scaled, X_test_scaled, Y_train_encoded, Y_test_encoded = pre_process(
        Features, Labels, train_index, test_index
    ) 
        
    train_feed = {X: X_train_scaled, Y: Y_train_encoded}
    test_feed = {X: X_test_scaled, Y: Y_test_encoded}
    
    train_acc, test_acc = run_training(train_feed, test_feed)
    train_accuracy += train_acc
    test_accuracy += test_acc
    
    print("Train accuracy:", train_acc)
    print("Test accuracy:", test_acc)
    print("")
    
train_accuracy /= 10
test_accuracy /= 10

train_writer.close()
test_writer.close()

print("")
print("Final train accuracy:", train_accuracy)
print("Final test accuracy:", test_accuracy)

Train accuracy: 0.9851852
Test accuracy: 1.0

Train accuracy: 0.9851852
Test accuracy: 0.8

Train accuracy: 1.0
Test accuracy: 0.8

Train accuracy: 0.9777778
Test accuracy: 1.0

Train accuracy: 0.9851852
Test accuracy: 0.93333334

Train accuracy: 0.9851852
Test accuracy: 0.93333334

Train accuracy: 0.9851852
Test accuracy: 1.0

Train accuracy: 0.9851852
Test accuracy: 0.8666667

Train accuracy: 0.9777778
Test accuracy: 1.0

Train accuracy: 0.9851852
Test accuracy: 1.0


Final train accuracy: 0.9851851999759674
Final test accuracy: 0.9333333373069763


## The End!